# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [3]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

all useful columns for this model are used as variables

In [4]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [5]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [6]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [7]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [8]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [9]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832163,0.102680,-37.321371,0.0,0.107574,-35.623351,0.0
asc_drive,-1.235707,0.073716,-16.763025,0.0,0.080400,-15.369411,0.0
asc_pt,-0.528767,0.053297,-9.921212,0.0,0.054766,-9.655006,0.0
beta_cost,-0.169780,0.013321,-12.745694,0.0,0.013108,-12.952753,0.0
beta_time,-5.450540,0.181898,-29.964846,0.0,0.202684,-26.891767,0.0


In [10]:
res.compileEstimationResults(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

In [ ]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [11]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [12]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [13]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.509593,0.171871,-26.238171,0.0,0.191028,-23.606920,0.0
asc_drive,-1.885701,0.106697,-17.673479,0.0,0.122740,-15.363361,0.0
asc_pt,-2.339709,0.114622,-20.412388,0.0,0.125798,-18.598938,0.0
beta_cost,-0.146061,0.014772,-9.887595,0.0,0.014850,-9.835754,0.0
beta_time_cycle,-6.121448,0.467854,-13.084093,0.0,0.542680,-11.280025,0.0
beta_time_drive,-6.445066,0.333687,-19.314718,0.0,0.391345,-16.469010,0.0
beta_time_pt,-3.514554,0.233212,-15.070204,0.0,0.258207,-13.611367,0.0
beta_time_walk,-8.204816,0.290737,-28.220736,0.0,0.364365,-22.518121,0.0


In [14]:
res.compileEstimationResults(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [18]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [19]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [20]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.766977,0.102770,-36.654510,0.000000,0.107734,-34.965548,0.000000
asc_drive,-1.137426,0.073820,-15.408139,0.000000,0.080194,-14.183498,0.000000
asc_pt,-0.831978,0.062068,-13.404320,0.000000,0.064856,-12.828129,0.000000
beta_cost_drive,-0.214644,0.017250,-12.443301,0.000000,0.021272,-10.090610,0.000000
beta_cost_pt,0.061554,0.026157,2.353272,0.018609,0.028466,2.162411,0.030587
beta_time,-5.411681,0.183127,-29.551508,0.000000,0.202913,-26.670010,0.000000


In [21]:
res.compileEstimationResults(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

### Comparaison

we compare the Akaike Information Criterion (AIC)

In [22]:
print("Model 0 PIC value :",results_generic.data.akaike)
print("Model 1, time alternative specification PIC value :",results_alt_spec_time.data.akaike)
print("Model 1, cost alternative specification PIC value :",results_alt_spec_cost.data.akaike)

Model 0 PIC value : 9142.89298347718
Model 1, time alternative specification PIC value : 8545.668496833014
Model 1, cost alternative specification PIC value : 9030.730461533794


we compare the Bayesian Information Criterion (BIC)

In [23]:
print("Model 0 PIC value :",results_generic.data.bayesian)
print("Model 1, alternative specification travel time, PIC value :",results_alt_spec_time.data.bayesian)
print("Model 1, alternative specification cost, PIC value :",results_alt_spec_cost.data.bayesian)

Model 0 PIC value : 9175.478949434262
Model 1, alternative specification travel time, PIC value : 8597.806042364344
Model 1, alternative specification cost, PIC value : 9069.833620682291


According to the AIC and the BIC, the best model is the one wih the alternative specific travel time coefficient, and the generic travel cost coefficient.

# Model 2

In [24]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [25]:
model_base=biogeme_model1

In [26]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [27]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [28]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [29]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.566487,0.174291,-26.200303,0.000000,0.196946,-23.186451,0.000000
asc_drive,-2.782434,0.119796,-23.226450,0.000000,0.138492,-20.090931,0.000000
asc_pt,-2.421769,0.118993,-20.352162,0.000000,0.131403,-18.430077,0.000000
beta_cost,-0.140456,0.014385,-9.763946,0.000000,0.013889,-10.112727,0.000000
beta_license,-0.517648,0.025889,-19.994674,0.000000,0.027952,-18.519361,0.000000
beta_pt_inter,-0.107553,0.087587,-1.227953,0.219464,0.089736,-1.198553,0.230702
beta_time_cycle,-6.305829,0.478522,-13.177721,0.000000,0.559566,-11.269139,0.000000
beta_time_drive,-7.003843,0.351689,-19.914879,0.000000,0.411151,-17.034718,0.000000
beta_time_pt,-3.541166,0.259076,-13.668426,0.000000,0.282515,-12.534425,0.000000
beta_time_walk,-8.376049,0.296657,-28.234810,0.000000,0.377032,-22.215757,0.000000


In [30]:
res.compileEstimationResults(results_model2)

AttributeError: module 'biogeme.results' has no attribute 'compileEstimationResults'

In [31]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [32]:
#we test them using the null hypothesis that our base model is the true model

In [33]:
results_base=model_base.estimate()

In [34]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

AttributeError: 'bioResults' object has no attribute 'likelihood_ratio_test'

In [35]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [36]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

5.991464547107979